<h1><center>Laboratorio 5: Benchmark Bayesiano 🧮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Diego Irarrázaval

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Joaquín Ortega
- Nombre de alumno 2: MAría José Zambrano


### **Link de repositorio de GitHub:** https://github.com/mzambrano1/Lab-programacion-cientifica

### Indice 

1. [Temas a tratar](#Temas-a-tratar:)
3. [Descripcción del laboratorio](#Descripción-del-laboratorio.)
4. [Objetivos principales del laboratorio](#Objetivos-principales-del-laboratorio)

# Temas a tratar

- Optimización de Código en Python.
- Utilización de librerías para medir el tiempo de ejecución de funciones.
- Métodos para optimizar el rendimiento de las funciones.

# Reglas:

- Fecha de entrega: 4/06/2021
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

# Objetivos principales del laboratorio

- Obtener datos desde Reddit y visualizar cuales post son más probables que sean puntuados positivamente.
- Aplicar un atajo bayesiano para obtener la mean posterior de datos.
- Optimizar a través de librerías funciones.

#Importamos librerias utiles 😸

In [5]:
%%capture
!pip install praw
!pip install line_profiler
!pip install memory_profiler
import sys
import praw
import time
import numpy as np
import pandas as pd
from numba import jit
import plotly.express as px
from functools import lru_cache
from IPython.core.display import Image
import plotly.graph_objects as go
import timeit

# 1. Recomendando Posts de Subreddit

<p align="center">
  <img src="https://styles.redditmedia.com/t5_2rer8/styles/bannerBackgroundImage_6o2td1zc54671.jpg?width=4000&format=pjpg&s=600bcf8560dff264f1cc7c0785ba5f6d529e0c28" width="10000">
</p>

Unos intrépidos alumnos del curso, quienes frecuentemente participan en subreddits y foros de reviews, se preguntan lo siguiente: ¿Podremos confiar que un post es bueno, si este tiene solamente 1 o 3 votos positivos?. los compañeros, creen que esto claramente no representa una opinión general, ya que estamos mucho menos seguros acerca de la verdadera proporción de votos a favor de los comentarios con pocos datos. ¿Pero cómo podemos obtener una representación más creíble para este problema?.

Lo señalado forma parte de un problema Bayesiano, donde a través del cálculo de la posterior se puede conocer que tan probable es que un post sea bueno. Para efectos de este laboratorio, no se exige un conocimiento previo para resolver este problema, simplemente se deberá aplicar las ecuaciones presentadas más adelante (De igual forma si quedan interesados sobre el tema se les invita a tomar el ramo [CC6104](https://github.com/dccuchile/CC6104)).

## 1.1 Obtención de Subrredits y Análisis

Para estudiar que tan probable es que un post sea positivo se comenzará cargando datos reales del subreddit chile desde reddit (si usted desea puede cambiar el subreddit a uno de su gusto). Para esto le proponemos la utilización de la función que aparece mas abajo, la que presenta un usuario ya creado por el equipo docente. Dese un tiempo para entender que hace cada parte de la función, visualizando que se obtiene de estas.

Revisada la función, utilice un **perfilador** para monitorear el tiempo y memoria que les toma a las subfunciones ser ejecutadas (por linea de código, vean las clases). Señale cuales son los procesos que mas tiempo consumen en la ejecución del Código, comentando si es posible mejorar el desempeño de la función.

**TO-DO:**
- [ ] Estudiar la función propuesta por el equipo docente.
- [ ] Estudiar los tiempos de ejecución del Código a través de un perfilador.

In [6]:
#%%file praw_reddit_module.py

def praw_reddit(nombre_subreddit = 'chile', n_hot = 1000):
  reddit = praw.Reddit(client_id='-w2hyFINxZ8T3g',
                     client_secret='zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw',
                     password='ClasesMDS7202',
                     user_agent='Clases',
                     username='DocenciaDataScience', check_for_async=False)
  subreddit  = reddit.subreddit(nombre_subreddit)

  votes, post, url = {}, {}, {}
  top_submissions = list(subreddit.hot(limit = n_hot))
  for it, top_n in enumerate(range(50, len(top_submissions),50)):
    top_n_submissions = top_submissions[:top_n]
    upvotes, downvotes, url[it], post[it] = [], [], [], []

    for submission in top_n_submissions:
        try:
            ratio = submission.upvote_ratio
            ups = int(round((ratio*submission.score)/(2*ratio - 1)) if ratio != 0.5 else round(submission.score/2))
            upvotes.append(ups)
            downvotes.append(ups - submission.score)
            post[it].append(submission.title)
            url[it].append(submission.url)
        except Exception as e:
            continue
    votes[it] = np.array([upvotes, downvotes]).T
  return votes, post, url

In [ ]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [ ]:
%lprun -f praw_reddit praw_reddit()

In [ ]:
!pip install line_profiler
%load_ext line_profiler
%lprun -f praw_reddit praw_reddit()

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [ ]:
from mod import praw_reddit #Trabjamos en colab y esta parte no nos funcionó :( la tuvimos que correr local
!pip install memory_profiler #EL warning es por si no llegase a correr (en todo caso debería pero por si las moscas)
%load_ext memory_profiler
%mprun -f praw_reddit praw_reddit()

# Tiempo de ejecución

Timer unit: 1e-06 s

Total time: 14.3682 s
File: <ipython-input-6-8fa850861273>
Function: praw_reddit at line 3

    Line #      Hits         Time  Per Hit   % Time  Line Contents

     3                                           def praw_reddit(nombre_subreddit = 'chile', n_hot = 1000):
     4         1          6.0      6.0      0.0    reddit = praw.Reddit(client_id='-w2hyFINxZ8T3g',
     5         1          2.0      2.0      0.0                       client_secret='zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw',
     6         1          3.0      3.0      0.0                       password='ClasesMDS7202',
     7         1          1.0      1.0      0.0                       user_agent='Clases',
     8         1       3711.0   3711.0      0.0                       username='DocenciaDataScience', check_for_async=False)
     9         1         68.0     68.0      0.0    subreddit  = reddit.subreddit(nombre_subreddit)
    10                                           
    11         1          2.0      2.0      0.0    votes, post, url = {}, {}, {}
    12         1   14258808.0 14258808.0     99.2    top_submissions = list(subreddit.hot(limit = n_hot))
    13        20         56.0      2.8      0.0    for it, top_n in enumerate(range(50, len(top_submissions),50)):
    14        19        463.0     24.4      0.0      top_n_submissions = top_submissions[:top_n]
    15        19        148.0      7.8      0.0      upvotes, downvotes, url[it], post[it] = [], [], [], []
    16                                           
    17      9519       8944.0      0.9      0.1      for submission in top_n_submissions:
    18      9500       7916.0      0.8      0.1          try:
    19      9500      16619.0      1.7      0.1              ratio = submission.upvote_ratio
    20      9500      18035.0      1.9      0.1              ups = int(round((ratio*submission.score)/(2*ratio - 1)) if ratio != 0.5 else round(submission.score/2))
    21      9500       9427.0      1.0      0.1              upvotes.append(ups)
    22      9500      10094.0      1.1      0.1              downvotes.append(ups - submission.score)
    23      9500      14762.0      1.6      0.1              post[it].append(submission.title)
    24      9500      16691.0      1.8      0.1              url[it].append(submission.url)
    25                                                   except Exception as e:
    26                                                       continue
    27        19       2407.0    126.7      0.0      votes[it] = np.array([upvotes, downvotes]).T
    28         1          1.0      1.0      0.0    return votes, post, url





**Respuesta:** En cuanto al tiempo de ejecución es posible notar que este es considerable, 14 segundos. Observando el análisis línea por línea destaca la ejecución de 

```
top_submissions = list(subreddit.hot(limit = n_hot))
```
esta instrucción se lleva aproximadamente un 99% del tiempo de ejecución de la función, es la parte más lenta que tiene esta función, lo cual tiene sentido y que es la parte encargada de realizar la request de los post a reddit.En caso de querer  querer optimizar la función lo más sensato sería partir por acá ya que es lo más lento que hay. 

Como esta parte depende de la respuesta de Reddit, no se tiene mucho control sobre el tiempo que demora, por lo que la mejor forma de optimzarla es guardar las respuestas en caché la primera vez que se ejecuta la función, de manera que las veces siguientes no se tenga que hacer la request y se tengan los resultados accesibles rapidamente

Aparte de la línea ya mencionada, lo que va dentro del for:

```
for submission in top_n_submissions:
        try:
            ratio = submission.upvote_ratio
            ups = int(round((ratio*submission.score)/(2*ratio - 1)) if ratio != 0.5 else round(submission.score/2))
            upvotes.append(ups)
            downvotes.append(ups - submission.score)
            post[it].append(submission.title)
            url[it].append(submission.url)
        except Exception as e:
            continue
```
es lo siguiente que más tiempo lleva. Estas son eperaciones que se repiten varias veces, el optimizar tan solo un poco estas línas podría tener un impacto notable en el tiempo de ejecución total.

Una forma de optimizar estas instrucciones es el uso de arreglos de Numpy, los cuales son más eficientes que las listas de python, y también el uso de defaultdict, los cuales también son más eficientes que los diccionarios nativos de python.


#Uso de memoria

    Line   Mem usage    Increment  Occurences   Line Contents

     4     67.3 MiB     67.3 MiB           1   def praw_reddit(nombre_subreddit = 'chile', n_hot = 1000):
     5     67.4 MiB      0.1 MiB           2     reddit = praw.Reddit(client_id='-w2hyFINxZ8T3g',
     6     67.3 MiB      0.0 MiB           1                        client_secret='zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw',
     7     67.3 MiB      0.0 MiB           1                        password='ClasesMDS7202',
     8     67.3 MiB      0.0 MiB           1                        user_agent='Clases',
     9     67.3 MiB      0.0 MiB           1                        username='DocenciaDataScience', check_for_async=False)
    10     67.4 MiB      0.0 MiB           1     subreddit  = reddit.subreddit(nombre_subreddit)
    11                                         
    12     67.4 MiB      0.0 MiB           1     votes, post, url = {}, {}, {}
    13     88.2 MiB     20.7 MiB           1     top_submissions = list(subreddit.hot(limit = n_hot))
    14     88.2 MiB      0.0 MiB          20     for it, top_n in enumerate(range(50, len(top_submissions),50)):
    15     88.2 MiB      0.0 MiB          19       top_n_submissions = top_submissions[:top_n]
    16     88.2 MiB      0.0 MiB          19       upvotes, downvotes, url[it], post[it] = [], [], [], []
    17                                         
    18     88.2 MiB      0.0 MiB        9519       for submission in top_n_submissions:
    19     88.2 MiB      0.0 MiB        9500           try:
    20     88.2 MiB      0.0 MiB        9500               ratio = submission.upvote_ratio
    21     88.2 MiB      0.0 MiB        9500               ups = int(round((ratio*submission.score)/(2*ratio - 1)) if ratio != 0.5 else round(submission.score/2))
    22     88.2 MiB      0.0 MiB        9500               upvotes.append(ups)
    23     88.2 MiB      0.0 MiB        9500               downvotes.append(ups - submission.score)
    24     88.2 MiB      0.0 MiB        9500               post[it].append(submission.title)
    25     88.2 MiB      0.0 MiB        9500               url[it].append(submission.url)
    26                                                 except Exception as e:
    27                                                     continue
    28     88.2 MiB      0.0 MiB          19       votes[it] = np.array([upvotes, downvotes]).T
    29     88.2 MiB      0.0 MiB           1     return votes, post, url


**Respuesta:** Con respecto al uso de memoria de la función es nuevamente nuestra querida línea 

```
20.7 MiB           1     top_submissions = list(subreddit.hot(limit = n_hot))
```
La responsable de extraer la información desde Reddit, la que se lleva el premio por el mayor uso de memoria. El que esto sea así tiene bastante sentido, ya que sse están guardando los posts de Reddit en esa línea, por lo que se necesita usar memoria para hacerlo. Aparte de esta línea no se aprecia realmente el uso de memoria en el resto del código de la función, más que en la parte donde se define la misma, porque claro, la función tiene que guardarse an alguna parte.
En caso de querer optimizar el uso de la memoria no hay mucho que hacer, ya que los datos deben ser guardados, si no la función pierde su gracia, la mejor opción para esto, al igual que en la parte anterior, es guardar los datos en el caché cosa de no tener que pedirls y guardarlos cada vez.
 

## 1.2 Análisis de Tiempo con Cache

Sin duda, un factor clave en la mejora del tiempo de ejecución de una aplicación es el uso eficiente de la memoria. Por lo que es importante, que ustedes respondan las siguientes preguntas:
1.	¿Qué es la memoria cache y a que se refiere las siglas LRU?
2.	¿Cuáles son los costos que tiene la aplicación de técnicas de Caching?
3. ¿Cuál es la consecuencía de ocupar caching en la función anterior?.

Respondidas las preguntas, se le solicita que aplique alguna técnica de caching para mejorar el desempeño de la función `praw_reddit`. Para esto compare solo el tiempo de ejecución del algoritmo con y sin caching, señalando el tiempo total de ejecución y el tiempo promedio que le toma ejecutar cada loop a la función. Con lo anterior, ¿es posible visualizar mejoras en este caso?.


**TO-DO:**
- [ ] Responder preguntas.
- [ ] Mejorar el código con cache.


**Respuestas Teóricas:**

1. La memoria caché es una memoria auxiliar que utiliza la CPU para almacenamiento temporal de datos recientemente procesados, para un rapido acceso.
LRU se refiere al algoritmo de memoria menos usada recientemente (*Least Recently Used*), y tal como su nombre lo dice, es una forma de administrar la memoria almacenada en caché, eliminando los datos que han sido menos usados recientemente, reemplazandolos por los que se quieren usar en el momento.

2. El costo que tiene usar estrategias de caching radica en el consumo de memoria, lo cual tiene un costo alto si la memoria está localizada en disco, ya que el acceso podría ser muy lento.

3. El uso de caching puede mejorar el tiempo si es que se guardan los datos que retorna Reddit la primera vez que se llama a la función, y se usan las siguientes veces que se llama. Esto pues se evita rehacer una consulta, que como se demostró anteriormente, es lo que más cuesta.

In [7]:
%timeit praw_reddit()

38.9 s ± 18.2 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
@lru_cache(maxsize=10000)
def praw_reddit(nombre_subreddit = 'chile', n_hot = 1000):
  reddit = praw.Reddit(client_id='-w2hyFINxZ8T3g',
                     client_secret='zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw',
                     password='ClasesMDS7202',
                     user_agent='Clases',
                     username='DocenciaDataScience', check_for_async=False)
  subreddit  = reddit.subreddit(nombre_subreddit)

  votes, post, url = {}, {}, {}
  top_submissions = list(subreddit.hot(limit = n_hot))
  for it, top_n in enumerate(range(50, len(top_submissions),50)):
    top_n_submissions = top_submissions[:top_n]
    upvotes, downvotes, url[it], post[it] = [], [], [], []

    for submission in top_n_submissions:
        try:
            ratio = submission.upvote_ratio
            ups = int(round((ratio*submission.score)/(2*ratio - 1)) if ratio != 0.5 else round(submission.score/2))
            upvotes.append(ups)
            downvotes.append(ups - submission.score)
            post[it].append(submission.title)
            url[it].append(submission.url)
        except Exception as e:
            continue
    votes[it] = np.array([upvotes, downvotes]).T
  return votes, post, url

In [10]:
%timeit praw_reddit()

The slowest run took 12.00 times longer than the fastest. This could mean that an intermediate result is being cached.
329 ns ± 357 ns per loop (mean ± std. dev. of 7 runs, 1 loop each)


Al ejecutar la función sin utilizar son caching, se tiene un tiempo pomedio de 38.9 s por loop, mientras que al utilizar caching se tiene un tiempor promedio por loop de 329 ns. Esto muestra que al utilizar caching se tiene una siginificativa mejora en el tiempo de ejecución, ya que es unas $10^8$ veces más rápida al utilizarlo.

Algo que vale la pena notar es que al utilizar caching la diferencia entre la ejecución más lenta y las más rápida es bastante grande, ya que la primera vez que se ejecuta la función es necesario que se ejecute normalmente y la las ejecuciones siguientes se aprovechan del guardado en cache de la ejecución anterior

En conclusión, el uso de cahching, en este caso, es realmente efectivo en disminuir el tiempo de ejecución



## 1.3 Obtención de Mean Posterior y Standard Error

Una forma de obtener la mean posterior y el Standard Error de los datos de reddit es aplicando un atajo de inferencia Bayesiana. Este atajo se define de la siguiente forma:

Sea **u** los votos positivos y **d** los votos negativos del subreddit, tendremos que:

$$a = 1+u$$
$$b = 1+d$$

$$\sigma= 1.65\sqrt(\dfrac{ab}{(a + b)^2(a + b + 1)})$$

$$\mu = \dfrac{a}{a+b}$$

Donde $\mu$ es la mean posterior y $\sigma$ el standard error.

Con lo anterior, genere dos funciones que tengan como salida $\mu$ y $\sigma$ de acuerdo a las ecuaciones señaladas. La primera función, deberá ser construida sin el uso de numpy, aplicando for y aplicando comandos nativos de Python. Por otro lado, deberá generar una segunda función con el uso exclusivo de numpy. **OJO** que las funciones deben tener como entrada solo un elemento del diccionario votes (por ejemplo `votes[1]`), por lo que estas no deben tener como entrada el conjunto completo de datos.

In [6]:

def intervalos(votos_positivos, votos_negativos):
  mu = [] 
  sigma = []
  for i in range(len(votos_positivos)):
    a = votos_positivos[i] + 1
    b = votos_negativos[i] + 1
    m = a/(a+b)
    mu.append(m)
    arriba = a*b
    cuadrado = (a+b)**2
    otro = a+b+1
    abajo = cuadrado*otro
    s =  1.65*((arriba/abajo)**(0.5))
    sigma.append(s)
  return mu,sigma
    


def intervalos_numpy(votos_positivos, votos_negativos):
  votos_positivos = votos_positivos + 1 
  votos_negativos = votos_negativos + 1
  mu = (votos_positivos)/(votos_positivos + votos_negativos)
  arriba = votos_positivos*votos_negativos
  cuadrado = (votos_positivos+votos_negativos)**2
  otro = votos_positivos+votos_negativos+1
  abajo = cuadrado*otro
  sigma = 1.65*np.sqrt(arriba/abajo)
  return mu, sigma


## 1.4 Comparación de rendimiento

Creadas las funciones de la sección 1.3, de forma iterativa compare los diferentes batches de datos que contiene el diccionario `votes`. Para esto genere un gráfico utilizando plotly, donde se pueda observar las diferencias de tiempo que toma ejecutar las diferentes cantidades de datos. ¿es posible observar una diferencia? ¿a qué se deberá esto?.

Aplique el compilador **Numba** sobre las funciones 1.3 y compare el tiempo de ejecución con los obtenidos sin el compilador. Grafique estos tiempos y observe comente los desempeños.

In [7]:
votes,post,url = praw_reddit()

In [19]:
#se obtienen los tiempos para cada elemento de votes
t = []
t_numpy = []
for i in votes.values():
  ts = t.append(timeit.timeit('[intervalos(i[:, 0], i[:, 1])]', globals=globals(),number=100)) 
  t_numpys = t_numpy.append(timeit.timeit('[intervalos_numpy(i[:, 0], i[:, 1])]', globals=globals(),number=100) )

n = [i.shape[0] for i in votes.values()] # se guardan la cantidad de datos de cada uno

In [20]:

fig = go.Figure()
fig.add_trace(go.Scatter(x=n, y=t,
                    mode='lines+markers',
                    name='Python puro'))
fig.add_trace(go.Scatter(x=n, y=t_numpy,
                    mode='lines+markers',
                    name='Numpy'))
fig.update_layout(title='Comparación del uso de Python solo y Numpy',
                   xaxis_title='Cantidad de datos',
                   yaxis_title='Tiempo [s]')

fig.show()

En el gráfico se puede observar la diferencia que existe, en tiempo de ejecución entre la función de python puro y la que utiliza numpy. Ambas empiezan con tiempos similares, pero la de python puro se comienza a disparar rápidamente a medida que se aumenta la cantidad de datos, mientras que la que utiliza numpy se mantiene casi constante a simple vista.

Esto se debe a que la librería Numpy se preocupa de realizar las operaciones más eficientemente que python, además de esto no se itera por sobre el input agregando cada resultado a una lista final, si no que se realizan las opreciones directamente sobre los vectores de entrada, lo que ahorra tiempo.

In [21]:

@jit(nopython=True)
def intervalos_JIT(votos_positivos, votos_negativos):
  mu = [] 
  sigma = []
  for i in range(len(votos_positivos)):
    a = votos_positivos[i] + 1
    b = votos_negativos[i] + 1
    m = a/(a+b)
    mu.append(m)
    arriba = a*b
    cuadrado = (a+b)**2
    otro = a+b+1
    abajo = cuadrado*otro
    s =  1.65*((arriba/abajo)**(0.5))
    sigma.append(s)
    return mu,sigma

@jit(nopython=True)
def intervalos_numpy_JIT(votos_positivos, votos_negativos):
  votos_positivos = votos_positivos + 1 
  votos_negativos = votos_negativos + 1
  mu = (votos_positivos)/(votos_positivos + votos_negativos)
  arriba = votos_positivos*votos_negativos
  cuadrado = (votos_positivos+votos_negativos)**2
  otro = votos_positivos+votos_negativos+1
  abajo = cuadrado*otro
  sigma = 1.65*np.sqrt(arriba/abajo)
  return mu, sigma

In [22]:
t_jit = []
t_numpy_jit = []
for i in votes.values():
  ts = t_jit.append(timeit.timeit('[intervalos_JIT(i[:, 0], i[:, 1])]', globals=globals(),number=100)) 
  t_numpys = t_numpy_jit.append(timeit.timeit('[intervalos_numpy_JIT(i[:, 0], i[:, 1])]', globals=globals(),number=100) )

In [23]:

fig = go.Figure()
fig.add_trace(go.Scatter(x=n, y=t,
                    mode='lines+markers',
                    name='Python puro'))
fig.add_trace(go.Scatter(x=n[1:], y=t_numpy[1:],
                    mode='lines+markers',
                    name='Numpy'))

fig.add_trace(go.Scatter(x=n[1:], y=t_jit[1:],
                    mode='lines+markers',
                    name='Python puro + Jit'))
fig.add_trace(go.Scatter(x=n[1:], y=t_numpy_jit[1:],
                    mode='lines+markers',
                    name='Numpy + Jit'))

fig.update_layout(title='Comparación de las funciones, con las mismas usando Jit',
                   xaxis_title='Cantidad de datos',
                   yaxis_title='Tiempo [s]')
fig.show()

En el gráfico anterior se muestran las comparaciones en tiempo de ejecución de las funciones ya implemantadas, comparandolas con ellas mismas, pero usando el compilador de Numba. Es posible notar como la función solo con Numpy tiene tiempo mucho mayores al resto, las cuales se encuentran muy juntas cercanas al cero y no se disintguen bien, por lo que se hace un 'Zoom':


In [24]:

fig = go.Figure()
fig.add_trace(go.Scatter(x=n[1:], y=t_numpy[1:],
                    mode='lines+markers',
                    name='Numpy'))

fig.add_trace(go.Scatter(x=n[1:], y=t_jit[1:],
                    mode='lines+markers',
                    name='Python + Jit'))
fig.add_trace(go.Scatter(x=n[1:], y=t_numpy_jit[1:],
                    mode='lines+markers',
                    name='Numpy + Jit'))

fig.update_layout(title='Acercamiento a la comparación del uso de Jit',
                   xaxis_title='Cantidad de datos',
                   yaxis_title='Tiempo [s]')
fig.show()

Acá se pueden apreciar de mejor manera la diferencia entre usar la función implementada con Numpy y las dos implementaciones con el compilador de Numba.

Estas últimas tienen tiempos de ejecución bastante menores, para todas las cantidades de datos, que la función que utiliza solo Numpy (y por transitividad también son mucho menores que la de python puro).

Esto muestra que la utilización del compilador es una buena de disminuir el tiempo que toma ejecutar ciertos códigos.

Una cosa interesante a notar es que el usar Jit optimiza de mejor manera el tiempo de ejecución de la función con python puro que la con numpy, siendo que se pensaría que esta última
siempre sería más eficiente que python puro.

## 1.5 Plot de Resultados Bayesianos

Llego la hora de visualizar los resultados obtenidos, para esto solo ejecute las siguientes celdas y observe lo que sucede :3. ¿Qué logra observar de los resultados?, ¿La solución resulta trivial?

En esta sección esperamos que solo comenten con lo que logran visualizar de los dataframes.

In [31]:
votos = votes[len(votes)-1]
posteo = post[len(post)-1]
print("lower bounds aproximados:")
posterior_mean, std_err = intervalos_numpy(votos[:, 0], votos[:, 1])
lb = posterior_mean - std_err
print("\nTop 20 post ordenador por el limite inferior:\n")
order = np.argsort(-lb)
vote_post = {'Votos (+)':votos[order[:20], 0], 'Votos (-)':votos[order[:20], 1] ,'Post':np.array(posteo)[order[:20]], 'url': np.array(url[len(url)-1])[order[:20]]}
df = pd.DataFrame(data=vote_post)
display(df)
ordered_post = df.Post

lower bounds aproximados:

Top 20 post ordenador por el limite inferior:



,Votos (+),Votos (-),Post,url
0,936,0,Gato andino extremadmente raro mirando una cám...,https://v.redd.it/k2r2bhlh0h971
1,720,0,"Debate presidencial 2005, pregunta clave de Ma...",https://v.redd.it/xj4g8rc5sw671
2,525,0,¿Qué hacer con Talca? Bueno...,https://i.redd.it/1tk589eq89771.png
3,460,0,"El bombardero de Lo Prado: ""Se lo mando seco e...",https://v.redd.it/l6c86bhd2w571
4,448,0,"Aysén hoy 2, no se subieron todas juntas (Bonu...",https://www.reddit.com/gallery/o6f7lr
5,402,0,"Juan Carlos Bodoque, This Charming Man (Video ...",https://v.redd.it/wf0yqt92tv771
6,358,0,Fotografía de un vendedor ambulante en la Plaz...,https://www.reddit.com/gallery/oc0qzt
7,345,0,Avión Pingüino Rey de Aerovías DAP que vuela e...,https://i.redd.it/f82si2h743771.jpg
8,271,0,Cuando el chico Paris y Siches eran compañeros...,https://v.redd.it/nkfiwmwgma571
9,250,0,Con 99% de aprobación: Bomberos es la instituc...,https://www.24horas.cl/nacional/bomberos-aprob...


In [32]:
r_order = order[::-1][-20:]
data_dic = {'mean':posterior_mean[r_order], 'std_err':std_err[r_order], 'post':ordered_post[::-1]}
df = pd.DataFrame(data=data_dic)
fig = px.scatter(df, x="mean", y="post",
                 error_x="std_err")
fig.show()

**Respuesta:**

En los resultados se muestran los posts que cuentan con mayor proporción de valoraciones positivas, osea los que podrían ser considerados como mejor valorados por los usuarios.

La manera en que se muestran los posts en el gráfico da a entender que están ordenados de los "más positivos" a los "menos positivos", con una descviación estándar creciente hasta cirte punto, luego comienza a ser más variable.

Esta desviación estándar parece estar relacionada con que tan cargado haacia un lado (positivo o negativo) está cada post.

Los resultados obtenidos no son triviales, ya que no resulta intuitivo el hecho que los posts con votos más cargados hacia un lado tengan la menor desviación, se tienede a pensar que mientras más variados sean los resultados mayor es la desviación.


# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana y que **los días de atraso no se pueden utilizar para entregas de lab, solo para tareas**. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media1.tenor.com/images/fb5bf7cc5a4acb91b4177672886a88ba/tenor.gif?itemid=5591338">
</p>